In [ ]:
import sys
sys.path.append('..')

In [ ]:
from copy import deepcopy
import random
from library.problems.ks import KSSolution
from library.problems.data.ks_data import weights, values, capacity
from library.algorithms.genetic_algorithms.algorithm import genetic_algorithm
from library.algorithms.genetic_algorithms.selection import fitness_proportionate_selection

## Genetic operators

### Binary Standard Mutation

This mutation operator is used for binary string or list representations, such as '10001' or [1, 0, 0, 0, 1], found in problems like the Knapsack or IntBin problems.

Standard binary mutation works by iterating over each position (or gene) in the binary string. For each gene, there is a fixed mutation probability that determines whether the bit should be flipped (a 0 becomes 1 and vice versa)

![Binary Standard Mutation](images/binary-std-mutation.png)

Let's implement a function for standard binary mutation. It takes a binary representation and a mutation probability as inputs and returns a new representation.


In [ ]:
def binary_standard_mutation(representation: str | list, mut_prob):
    # TODO
    pass

Let's test on the Knapsack problem:

In [ ]:
solution = KSSolution(values=values, weights=weights, capacity=capacity)

print("Solution:", solution)

new_solution_repr = binary_standard_mutation(solution.repr, mut_prob=0.2)

print("New solution:", new_solution_repr)

### Swap mutation

Swap mutation is a general-purpose operator suitable for any type of string or list-based representation.

It works by randomly selecting two positions (genes) in the solution and swapping their values. This swap is applied with a certain mutation probability.

![Swap Mutation](images/swap-mutation.png)

Let's implement the swap mutation function. It takes a representation and a mutation probability as inputs and returns a new solution where two genes may have been swapped.

In [ ]:
def swap_mutation(representation: str | list, mut_prob):
    # TODO
    pass

Now let's test on the Knapsack problem.

In [ ]:
solution = KSSolution(values=values, weights=weights, capacity=capacity)

print("Solution:", solution)

new_solution_repr = swap_mutation(solution.repr, mut_prob=0.8)

print("New solution:", new_solution_repr)

### Standard Crossover

Standard crossover takes two parent solutions, randomly selects a crossover point (an index between two consecutive genes) and exchanges the tail segments of the parents at that point. This process produces two new offspring that are combinations of their parents’ genetic material.

![Standard Crossover](images/std-crossover.png)

Let's implement the standard crossover function. It takes two parent representations as input and returns two offspring representations created by recombining segments from the parents at a randomly chosen crossover point.

In [ ]:
def standard_crossover(parent1_repr, parent2_repr):
   # TODO
   pass

Now let's test on the Knapsack problem.

In [ ]:
parent1 = KSSolution(values=values, weights=weights, capacity=capacity)
parent2 = KSSolution(values=values, weights=weights, capacity=capacity)

print("Parent 1:", parent1)
print("Parent 2:", parent2)

offspring1_repr, offspring2_repr = standard_crossover(parent1.repr, parent2.repr)

print("Offspring 1:", offspring1_repr)
print("Offspring 2:", offspring2_repr)

## Solving Knapsack with Genetic Algorithms

![Knapsack Solutions](images/ks-solutions.png)

In the last notebook we implemented the genetic algorithm function. This function receives the following arguments:
- `initial_population`: List of individuals (randomly generated solutions)
- `max_gen`: Maximum number of generations
- `selection_algorithm`: A function that receives a population, selects one individual based on fitness and returns it
- `maximization`: Boolean that indicates if we're solving a maximization or minimization problem
- `xo_prob`: Probability of crossover (usually big)
- `mut_prob`: Probability of mutation (usually small)
- `elistism`: A boolean that indicates if elitism should be used or not

For this function to work, we need to comply with some assumptions
- individuals have `fitness`, `crossover` and `mutation` methods
- `crossover` always returns two offspring
- both `crossover` and `mutation` methods return new individuals instead of modifying individuals in-place

To solve the Knapsack Problem (KS) using this GA framework, we can define a new class, `KSGASolution`, which extends `KSSolution`. This allows us to inherit methods like `fitness`, `random_initial_representation`, and the `repr` attribute.

In `KSGASolution`, we'll implement the required `crossover` and `mutation` methods, adhering to the above assumptions.

For simplicity, let's use the standard crossover and binary standard mutation.

In [ ]:
class KSGASolution(KSSolution):
    # TODO
    pass

Or we could just the functions we implemented in the beginning of the notebook!

In [ ]:
class KSGASolution(KSSolution):
    # TODO
    pass

#### How can I test different crossover and mutation operators?

There are two approaches:
- Create separate classes for each combination of crossover and mutation operators.

    For example:
    - `KS_StdXO_StdMut_GASolution`
    - `KS_StdXO_SwapMut_GASolution`

    This method works but can quickly become repetitive and hard to maintain as the number of combinations grows.

- Make the solution class accept crossover and mutation functions as parameters during initialization.

    These functions would operate directly on the internal representation of the individual.
    ✅ This approach is much more modular and flexible!
    You can easily swap operators without needing to define new classes each time, making experimentation and tuning much easier.

In [ ]:
class KSGASolution(KSSolution):
    def __init__(
        self,
        values,
        weights,
        capacity,
        mutation_function, # Callable
        crossover_function, # Callable
        repr = None
    ):
        super().__init__(
            values=values,
            weights=weights,
            capacity=capacity,
            repr=repr,
        )

        # Save as attributes for access in methods
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function

    
    def mutation(self, mut_prob):
        # Apply mutation function to representation
        new_repr = self.mutation_function(self.repr, mut_prob)
        # Create and return individual with mutated representation
        return KSGASolution(
            values=self.values,
            weights=self.weights,
            capacity=self.capacity,
            mutation_function=self.mutation_function,
            crossover_function=self.crossover_function,
            repr=new_repr
        )

    def crossover(self, other_solution):
        # Apply crossover function to self representation and other solution representation
        offspring1_repr, offspring2_repr = self.crossover_function(self.repr, other_solution.repr)

        # Create and return offspring with new representations
        return (
            KSGASolution(
                values=self.values,
                weights=self.weights,
                capacity=self.capacity,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring1_repr
            ),
            KSGASolution(
                values=self.values,
                weights=self.weights,
                capacity=self.capacity,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring2_repr
            )
        )

Let's test.

In [ ]:
repr = [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1]

# Using std crossover and std mutation
solution1 = KSGASolution(
    values=values,
    weights=weights,
    capacity=capacity,
    mutation_function=binary_standard_mutation,
    crossover_function=standard_crossover,
    repr=repr
)


# Using std crossover and swap mutation
solution2 = KSGASolution(
    values=values,
    weights=weights,
    capacity=capacity,
    mutation_function=swap_mutation,
    crossover_function=standard_crossover,
    repr=repr
)

In [ ]:
# Apply standard binary mutation
solution1.mutation(mut_prob=0.2)

In [ ]:
# Apply standard swap mutation
solution2.mutation(mut_prob=0.2)

### Apply genetic algorithm

Let's run the genetic algorithm to solve Knapsack using standard crossover and standard binary mutation

In [ ]:
# TODO: Apply genetic algorithm to KS using standard crossover and binary standard mutation

And finally, let's apply the genetic algorithm again, but this time using swap mutation with a higher probability since it is not as disruptive as standard binary mutation.

In [ ]:
# TODO: Apply genetic algorithm to KS using standard crossover and swap mutation